In [24]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")

In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
train_df = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
valid_df = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
label_to_index = ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]
train_sentence_counter, valid_sentence_counter = 0, 0
with open("../Data/Interim/data_v5_split.jsonl", "r") as f:
    for line in f:
        docData = json.loads(line)
        if docData[3]=="train":
            for w_i, word in enumerate(docData[0]):
                train_df["sentence_id"].append(train_sentence_counter)
                train_df["words"].append(word)
                train_df["labels"].append("{}".format(docData[1][w_i]))
                if docData[1][w_i] != "O":
                    label_index = label_to_index.index(docData[1][w_i])
                    train_df["labels"][-1] += "_{}".format(docData[2][label_index])
                    
            train_sentence_counter += 1
        else:
            for w_i, word in enumerate(docData[0]):
                valid_df["sentence_id"].append(valid_sentence_counter)
                valid_df["words"].append(word)
                valid_df["labels"].append("{}".format(docData[1][w_i]))
                if docData[1][w_i] != "O":
                    label_index = label_to_index.index(docData[1][w_i])
                    valid_df["labels"][-1] += "_{}".format(docData[2][label_index])
            valid_sentence_counter += 1
train_df = pd.DataFrame(train_df)
valid_df = pd.DataFrame(valid_df)

In [3]:
train_df["labels"].unique()

array(['A_13', 'O1_9', 'O', 'R1_1', 'O2_13', 'A_9', 'O1_6', 'O2_9',
       'A_10', 'O1_14', 'R1_2', 'O2_4', 'A_6', 'O1_11', 'O2_3', 'O1_10',
       'R1_5', 'O2_11', 'A_7', 'O1_5', 'R1_4', 'O1_8', 'R1_3', 'O2_7',
       'O2_6', 'O1_2', 'O1_7', 'O1_13', 'O2_5', 'O2_14', 'O2_10', 'O2_1',
       'O2_12', 'O1_4', 'O1_3', 'O2_2', 'A_8', 'O1_1', 'O1_12', 'O2_8',
       'A_3', 'D_6', 'A_2', 'D_3', 'D_4', 'D_7', 'D_1', 'D_8', 'D_2',
       'D_5', 'A_4', 'M_4', 'M_8', 'M_9', 'M_7', 'M_11', 'M_3', 'M_1',
       'M_5', 'M_12', 'M_10', 'M_2', 'M_6', 'A_5', 'DH_12', 'A_0', 'A_1',
       'A_11', 'A_12', 'S_3', 'S_2', 'S_1', 'S_4', 'G_1', 'R2_5', 'O3_7',
       'R2_1', 'O3_8', 'O3_5', 'R2_2', 'O3_13', 'O3_3', 'R2_3', 'R2_4',
       'O3_6', 'O3_14', 'O3_12', 'O3_1', 'O3_9', 'O3_10', 'O3_4', 'O3_11',
       'O3_2', 'N_1', 'DH_1', 'DH_16', 'DH_15', 'DH_22', 'DH_21', 'DH_17',
       'DH_6', 'DH_7', 'DH_8', 'DH_5', 'DH_11', 'DH_10', 'DH_19', 'DH_20',
       'DH_2', 'DH_14', 'DH_23', 'DH_18', 'DH_13', 'DH_3

In [4]:
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/Data/Interim/labels_names.json", "r") as f:
    labels_names = json.load(f)
number_of_sublabels = {}
for k, label in zip(['action', 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze'],
                   ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]):
    number_of_sublabels[label] = len(labels_names[k])

In [5]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 20
model_args.output_dir = "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/CommandParser_tokenCLS_ipynb_data5"

model = NERModel(
    "bert", 
    "/s/ls4/users/grartem/HuggingFace/models/rubert-tiny2",
    #"/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb/checkpoint-20664-epoch-9/",
    args=model_args,
    labels=["O"] + ["A_{}".format(i) for i in range(0, number_of_sublabels["A"])] + ["{}_{}".format(label, i) for label in ["D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"] for i in range(1, number_of_sublabels[label])]
)

Some weights of the model checkpoint at /s/ls4/users/grartem/HuggingFace/models/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassific

In [30]:
# Train the model
model.train_model(train_df, eval_data=valid_df)

  0%|          | 0/29 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/5835 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1686 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A_9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O1_13 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: R1_2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O2_8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A_7 seems not 

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O1_15 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O1_16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


(116700,
 defaultdict(list,
             {'global_step': [2000,
               4000,
               5835,
               6000,
               8000,
               10000,
               11670,
               12000,
               14000,
               16000,
               17505,
               18000,
               20000,
               22000,
               23340,
               24000,
               26000,
               28000,
               29175,
               30000,
               32000,
               34000,
               35010,
               36000,
               38000,
               40000,
               40845,
               42000,
               44000,
               46000,
               46680,
               48000,
               50000,
               52000,
               52515,
               54000,
               56000,
               58000,
               58350,
               60000,
               62000,
               64000,
               64185,
               6

# Test
надо посчитать отдельно для лейбла и класса

In [25]:
from RobotCommandClassifier import utils
from RobotCommandClassifier import *

In [6]:
model = NERModel(
    "bert", 
    "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/CommandParser_tokenCLS_ipynb_data5/checkpoint-116700-epoch-20/",
    #"/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb/checkpoint-20664-epoch-9/",
    args=model_args,
    labels=["O"] + ["A_{}".format(i) for i in range(0, number_of_sublabels["A"])] + ["{}_{}".format(label, i) for label in ["D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"] for i in range(1, number_of_sublabels[label])]
)

In [37]:
result = model.eval_model(valid_df)
result[0]

  0%|          | 0/27 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1686 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A_9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O1_13 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: R1_2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O2_8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A_7 seems not 

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O1_16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [7]:
valid_texts = []
with open("../Data/Interim/data_v5_split.jsonl", "r") as f:
    for line in f:
        docData = json.loads(line)
        if docData[3]=="valid":
            valid_texts.append(docData)

In [8]:
predictions, raw_outputs = model.predict([" ".join(x[0]) for x in valid_texts])

  0%|          | 0/27 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1686 [00:00<?, ?it/s]

In [9]:
from sklearn.metrics import classification_report
true_labels, pred_labels = [], []
for i in range(len(valid_texts)):
    true_labels.extend(valid_texts[i][1])
    pred_labels.extend([list(x.values())[0].split("_")[0] for x in predictions[i]])
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           A       1.00      1.00      1.00     21225
           D       1.00      1.00      1.00      4940
          DH       1.00      1.00      1.00      2614
           G       1.00      1.00      1.00      2156
           M       1.00      1.00      1.00      4303
           N       1.00      1.00      1.00      1364
           O       1.00      1.00      1.00     17427
          O1       1.00      1.00      1.00     13809
          O2       1.00      1.00      1.00      4882
          O3       1.00      1.00      1.00      2170
          R1       1.00      1.00      1.00      6270
          R2       1.00      1.00      1.00      2849
           S       1.00      1.00      1.00      1143

    accuracy                           1.00     85152
   macro avg       1.00      1.00      1.00     85152
weighted avg       1.00      1.00      1.00     85152



In [28]:
valid_array = np.array([x[2] for x in valid_texts])
valid_array = pd.DataFrame(valid_array, columns=['action', 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze'])

In [21]:
pred_array = []
for i in range(len(valid_texts)):
    y = np.zeros(len(label_to_index))
    for word in predictions[i]:
        word_label = list(word.values())[0]
        if word_label=="O":
            continue
        word_label =  word_label.split("_")        
        if y[label_to_index.index(word_label[0])]==0:
            y[label_to_index.index(word_label[0])] = int(word_label[1])
    pred_array.append(y)
pred_array = np.array(pred_array)

In [40]:
test_df

,x,y_name,y,action,direction,meters,degshours,object1,nearest,relation1,object2,relation2,object3,self,gaze,type,subset,fold
55828,постереги,simple,0,0,0,0,0,0,0,0,0,0,0,0,0,students,test,0
55829,дай по тормозам,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students,test,0
55830,ваша остановка,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students,test,0
55831,припаркуйся,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students,test,0
55832,приехали,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students,test,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57141,возле лесо стоит человек за лесом есть повален...,relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource,test,4
57142,отправляйся к спиленному деревуза лесополосой ...,relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource,test,4
57143,шагай в сторону дерева лежащего позади леса ко...,relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource,test,4
57144,рядом с человеком есть лес за которым поваленн...,relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource,test,4


array(['постереги', 'дай по тормозам', 'ваша остановка', ...,
       'шагай в сторону дерева лежащего позади леса который рядом с человеком ',
       'рядом с человеком есть лес за которым поваленное дерево подойди к этому дереву ',
       'рядом с человеком есть лес за лесом поваленное дерево иди к этому дереву'],
      dtype=object)

In [52]:
# If on test
test_df = pd.read_csv("../Data/Interim/one_cmd_v4_05072022_low_split.csv")
test_df = test_df[test_df["subset"]=="test"]
test_df = test_df[test_df["fold"]==4]
valid_array = test_df.loc[:,['action', 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']]

predictions, raw_outputs = model.predict(test_df["x"].values)

pred_array = []
for i in range(len(test_df)):
    y = np.zeros(len(label_to_index))
    for word in predictions[i]:
        word_label = list(word.values())[0]
        if word_label=="O":
            continue
        word_label =  word_label.split("_")        
        if y[label_to_index.index(word_label[0])]==0:
            y[label_to_index.index(word_label[0])] = int(word_label[1])
        else:
            if y[label_to_index.index(word_label[0])] != int(word_label[1]):
                print(i,"different labels for ", word_label[0])
    pred_array.append(y)
pred_array = np.array(pred_array)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/33 [00:00<?, ?it/s]

6 different labels for  A
8 different labels for  A
8 different labels for  A
29 different labels for  O1
30 different labels for  O1
31 different labels for  O1
33 different labels for  O1
37 different labels for  O1
41 different labels for  A
41 different labels for  O1
45 different labels for  A
48 different labels for  O1
50 different labels for  O1
51 different labels for  A
52 different labels for  O1
57 different labels for  O1
62 different labels for  R1
66 different labels for  O2
73 different labels for  O2
74 different labels for  O3
74 different labels for  O3
76 different labels for  R2
76 different labels for  R2
77 different labels for  O2
79 different labels for  O1
79 different labels for  R1
79 different labels for  O2
79 different labels for  R2
79 different labels for  R2
79 different labels for  R2
81 different labels for  O2
86 different labels for  O3
88 different labels for  O3
89 different labels for  O3
91 different labels for  O2
92 different labels for  O2
9

In [49]:
result = utils.calculate_metrics_2(valid_array, pred_array, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(valid_array, pred_array, config={
    "report_metrics": ["correct_samples_perc", "avg_macro_f1", "avg_acc", "class_acc", "class_macro_f1"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       0.17      0.30      0.21        10
         1.0       0.00      0.00      0.00         0
         2.0       0.50      0.50      0.50         8
         3.0       1.00      0.10      0.18        10
         5.0       0.00      0.00      0.00         0
         6.0       0.99      0.86      0.92       235
         7.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         0
        12.0       0.00      0.00      0.00         0
        13.0       0.00      0.00      0.00         0

    accuracy                           0.79       263
   macro avg       0.27      0.18      0.18       263
weighted avg       0.94      0.79      0.85       263

direction
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       245
         1.0       0.00      0.00      0.00         5
         2.0       0.00      0.00      0.00         3
       

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

39.0 	 correct_samples_perc
79.0 	 [action]_acc
18.0 	 [action]_macrof1
96.0 	 [direction]_acc
31.0 	 [direction]_macrof1
98.0 	 [meters]_acc
33.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
50.0 	 [degshours]_macrof1
68.0 	 [object1]_acc
33.0 	 [object1]_macrof1
98.0 	 [nearest]_acc
49.0 	 [nearest]_macrof1
81.0 	 [relation1]_acc
62.0 	 [relation1]_macrof1
76.0 	 [object2]_acc
48.0 	 [object2]_macrof1
78.0 	 [relation2]_acc
59.0 	 [relation2]_macrof1
76.0 	 [object3]_acc
52.0 	 [object3]_macrof1
95.0 	 [self]_acc
19.0 	 [self]_macrof1
86.0 	 [gaze]_acc
46.0 	 [gaze]_macrof1
42.0 	 avg_macro_f1
86.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu